In [17]:
# 📦 Step 1: Install libraries

# 📚 Step 2: Import packages
import os
import openai
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
from dotenv import load_dotenv
import json
import re

In [18]:
# 🔑 Step 3: Load API keys
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ALPHAVANTAGE_API_KEY = os.getenv("ALPHAVANTAGE_API_KEY") 

In [22]:
import os
import re
import json
import openai
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
from dotenv import load_dotenv

# 🔑 Load API keys
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
ALPHAVANTAGE_API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

# 🛠 Setup Alpha Vantage functions
def get_timeseries_data(ticker, dataset_type, start=None, end=None, interval="60min"):
    ts = TimeSeries(key=ALPHAVANTAGE_API_KEY, output_format='pandas')
    dataset_type = dataset_type.lower()
    
    if dataset_type == "daily":
        data, meta = ts.get_daily(symbol=ticker, outputsize='full')
    elif dataset_type == "intraday":
        data, meta = ts.get_intraday(symbol=ticker, interval=interval, outputsize='full')
    elif dataset_type == "weekly":
        data, meta = ts.get_weekly(symbol=ticker)
    elif dataset_type == "monthly":
        data, meta = ts.get_monthly(symbol=ticker)
    else:
        raise ValueError(f"Unsupported dataset type: {dataset_type}")

    # ✅ FIX: Sort index before slicing
    data = data.sort_index()

    if start or end:
        data = data.loc[start:end]
    
    return data


def get_fundamental_data(ticker, data_type):
    fd = FundamentalData(key=ALPHAVANTAGE_API_KEY, output_format='pandas')
    data_type = data_type.lower()
    
    if data_type == "income_statement":
        data, meta = fd.get_income_statement_annual(symbol=ticker)
    elif data_type == "balance_sheet":
        data, meta = fd.get_balance_sheet_annual(symbol=ticker)
    elif data_type == "overview":
        data, meta = fd.get_company_overview(symbol=ticker)
    else:
        raise ValueError(f"Unsupported fundamental type: {data_type}")
    
    return data

# 🤖 Parse prompt with OpenAI (keep your style)
def parse_prompt_with_llm(prompt):
    system_prompt = """
You are an expert finance assistant. 
When given a user's request about stock prices or fundamentals,
you must return ONLY valid JSON with these fields:

{
  "type": "timeseries" or "fundamental",
  "dataset_type": one of "daily", "weekly", "intraday", "income_statement", "balance_sheet", "overview",
  "ticker": "Ticker symbol",
  "start_date": "YYYY-MM-DD" (optional),
  "end_date": "YYYY-MM-DD" (optional),
  "interval": "15min" (optional, only for intraday)
}

IMPORTANT:
- Do NOT add explanations, markdown, comments, or formatting.
- Only raw JSON object.
- No ```json or ``` markers.
- Strict JSON output.
"""
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )

    content = response.choices[0].message.content

    # Safely remove any code block if LLM still tries to add it
    content = re.sub(r"^```(?:json)?\n*", "", content.strip())
    content = re.sub(r"\n*```$", "", content.strip())
    content = content.strip()

    try:
        parsed = json.loads(content)
    except json.JSONDecodeError:
        print(f"Bad LLM output: {content}")
        raise

    # ✅ Make sure 'type' exists
    if "type" not in parsed:
        raise ValueError(f"Invalid LLM response, missing 'type' field. Full output:\n{parsed}")
    
    return parsed

# 🚀 Fetch data from prompt
def fetch_data_from_prompt(prompt):
    parsed = parse_prompt_with_llm(prompt)
    
    print("Parsed prompt:", parsed)

    if parsed['type'] == 'timeseries':
        data = get_timeseries_data(
            ticker=parsed['ticker'],
            dataset_type=parsed['dataset_type'],
            start=parsed.get('start_date'),
            end=parsed.get('end_date'),
            interval=parsed.get('interval', '60min')
        )
    elif parsed['type'] == 'fundamental':
        data = get_fundamental_data(
            ticker=parsed['ticker'],
            data_type=parsed['dataset_type']
        )
    else:
        raise ValueError(f"Unknown type: {parsed['type']}")
    
    return data


In [26]:
# ✏️ Step 7: Try it!

user_prompt = "Show me Tesla's daily prices from 2022-12-01 to 2023-04-01"
data = fetch_data_from_prompt(user_prompt)

data.head(10)

Parsed prompt: {'type': 'timeseries', 'dataset_type': 'daily', 'ticker': 'TSLA', 'start_date': '2022-12-01', 'end_date': '2023-04-01'}


,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2022-12-01,197.080,198.92,191.800,194.70,80046213.0
2022-12-02,191.780,196.25,191.110,194.86,73645922.0
2022-12-05,189.440,191.27,180.550,182.45,93122667.0
2022-12-06,181.215,183.65,175.330,179.82,92150823.0
2022-12-07,175.030,179.38,172.218,174.04,84213284.0
2022-12-08,172.200,175.20,169.060,173.44,97624491.0
2022-12-09,173.840,182.50,173.360,179.05,104872336.0
2022-12-12,176.100,177.37,167.520,167.82,109794471.0
2022-12-13,174.870,175.05,156.910,160.95,175862722.0


In [27]:
# ----------------------------------------
# 📦 Example Prompts to Fetch Data
# ----------------------------------------

example_prompts = [
    "Show me Tesla's daily prices from 2023-01-01 to 2023-04-01",                    # daily time series
    "Give me Microsoft's weekly prices for 2022",                                     # weekly time series
    "Fetch Amazon's monthly closing prices for last year",                            # monthly time series
    "Show me Apple's intraday prices (15min interval) from 2024-01-01 to 2024-01-10",  # intraday
    "Get me Google's income statement",                                               # fundamental income statement
    "Retrieve Netflix's balance sheet",                                                # fundamental balance sheet
    "Give me an overview of Meta Platforms",                                           # company overview
    "Show me Nvidia's intraday prices (30min interval) from 2024-02-01 to 2024-02-02", # intraday with custom interval
    "Fetch Microsoft's income statement",                                              # another income statement
    "Give me Apple's balance sheet and financial summary"                             # another balance sheet
]

# ----------------------------------------
# 🧠 Loop through and fetch each dataset
# ----------------------------------------

for idx, prompt in enumerate(example_prompts):
    print(f"\n\n⚡ Prompt {idx+1}: {prompt}")
    try:
        data = fetch_data_from_prompt(prompt)
        print(f"✅ Data pulled successfully. Showing top 5 rows:")
        display(data.head())
    except Exception as e:
        print(f"❌ Error occurred: {e}")




⚡ Prompt 1: Show me Tesla's daily prices from 2023-01-01 to 2023-04-01
Parsed prompt: {'type': 'timeseries', 'dataset_type': 'daily', 'ticker': 'TSLA', 'start_date': '2023-01-01', 'end_date': '2023-04-01'}
✅ Data pulled successfully. Showing top 5 rows:


,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2023-01-03,118.47,118.80,104.6400,108.10,231402818.0
2023-01-04,109.11,114.59,107.5200,113.64,180388976.0
2023-01-05,110.51,111.75,107.1601,110.34,157986324.0
2023-01-06,103.00,114.39,101.8100,113.06,220911051.0
2023-01-09,118.96,123.52,117.1100,119.77,190283951.0




⚡ Prompt 2: Give me Microsoft's weekly prices for 2022
Parsed prompt: {'type': 'timeseries', 'dataset_type': 'weekly', 'ticker': 'MSFT', 'start_date': '2022-01-01', 'end_date': '2022-12-31'}
✅ Data pulled successfully. Showing top 5 rows:


,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2022-01-07,335.350,338.0000,310.09,314.04,174012812.0
2022-01-14,309.485,323.4100,303.75,310.20,193141391.0
2022-01-21,304.070,313.9093,295.61,296.03,180568678.0
2022-01-28,292.200,308.5000,276.05,308.26,352537899.0
2022-02-04,308.950,315.1200,299.96,305.94,202857381.0




⚡ Prompt 3: Fetch Amazon's monthly closing prices for last year
Parsed prompt: {'type': 'timeseries', 'dataset_type': 'monthly', 'ticker': 'AMZN', 'start_date': '2022-01-01', 'end_date': '2022-12-31'}
✅ Data pulled successfully. Showing top 5 rows:


...

,1. open,2. high,3. low,4. close,5. volume
date,,,,,




⚡ Prompt 9: Fetch Microsoft's income statement
Parsed prompt: {'type': 'fundamental', 'dataset_type': 'income_statement', 'ticker': 'MSFT'}
✅ Data pulled successfully. Showing top 5 rows:


,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
date,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.000000000,2024-06-30,USD,171008000000,245122000000,74114000000,74114000000,109433000000,7609000000,29510000000,61575000000,...,None,22287000000,107787000000,19651000000,None,88136000000,None,110722000000,133009000000,88136000000
1970-01-01 00:00:00.000000001,2023-06-30,USD,146052000000,211915000000,65863000000,65863000000,88523000000,7575000000,27195000000,57529000000,...,None,13861000000,89311000000,16950000000,None,72361000000,None,91279000000,105140000000,72361000000
1970-01-01 00:00:00.000000002,2022-06-30,USD,135620000000,198270000000,62650000000,62650000000,83383000000,5900000000,24512000000,52237000000,...,None,14460000000,83716000000,10978000000,None,72738000000,None,85779000000,100239000000,72738000000
1970-01-01 00:00:00.000000003,2021-06-30,USD,115856000000,168088000000,52232000000,52232000000,69916000000,5107000000,20716000000,45940000000,...,None,11686000000,71102000000,9831000000,None,61271000000,None,73448000000,85134000000,61271000000
1970-01-01 00:00:00.000000004,2020-06-30,USD,96937000000,143015000000,46078000000,46078000000,52959000000,5111000000,19269000000,43978000000,...,None,12796000000,53036000000,8755000000,None,44281000000,None,55627000000,68423000000,44281000000




⚡ Prompt 10: Give me Apple's balance sheet and financial summary
Parsed prompt: {'type': 'fundamental', 'dataset_type': 'balance_sheet', 'ticker': 'AAPL'}
✅ Data pulled successfully. Showing top 5 rows:


,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
date,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.000000000,2024-09-30,USD,364980000000,152987000000,29943000000,29943000000,7286000000,None,211993000000,None,...,20879000000,None,106629000000,50071000000,36634000000,56950000000,None,-19154000000,83276000000,15408095000
1970-01-01 00:00:00.000000001,2023-09-30,USD,352583000000,143566000000,29965000000,29965000000,6331000000,None,209017000000,43715000000,...,15807000000,None,111947000000,58829000000,49848000000,62146000000,None,-214000000,73812000000,15812547000
1970-01-01 00:00:00.000000002,2022-09-30,USD,352755000000,135405000000,23646000000,23646000000,4946000000,None,217350000000,42117000000,...,21110000000,None,120881000000,60845000000,49142000000,50672000000,None,-3068000000,64849000000,16325819000
1970-01-01 00:00:00.000000003,2021-09-30,USD,351002000000,134836000000,34940000000,34940000000,6580000000,None,216166000000,39440000000,...,15613000000,None,124719000000,47493000000,28636000000,63090000000,None,5562000000,57365000000,16864919000
1970-01-01 00:00:00.000000004,2020-09-30,USD,323888000000,143713000000,38016000000,38016000000,4061000000,None,180175000000,36766000000,...,13769000000,None,112436000000,42684000000,26320000000,65339000000,None,14966000000,50779000000,17528214000


In [28]:
# ----------------------------------------
# 📦 Additional ESG Prompts
# ----------------------------------------

esg_prompts = [
    "Fetch the ESG scores for Tesla",
    "Get me Microsoft's ESG disclosure metrics",
    "Show me Apple's environmental, social, and governance scores",
    "Retrieve Meta's ESG rating data",
    "Give me Google's ESG report details",
    "Show me Amazon's ESG factors summary",
    "Fetch the full ESG metrics of Nvidia",
    "What is the ESG performance of Netflix?",
    "Retrieve Salesforce's ESG scorecard",
    "Get the latest ESG ratings for Adobe"
]


In [30]:
# ----------------------------------------
# 🧠 Loop through and fetch each dataset
# ----------------------------------------

for idx, prompt in enumerate(esg_prompts):
    print(f"\n\n⚡ Prompt {idx+1}: {prompt}")
    try:
        data = fetch_data_from_prompt(prompt)
        print(f"✅ Data pulled successfully. Showing top 5 rows:")
        display(data.head())
    except Exception as e:
        print(f"❌ Error occurred: {e}")



⚡ Prompt 1: Fetch the ESG scores for Tesla
Parsed prompt: {'type': 'fundamental', 'dataset_type': 'overview', 'ticker': 'TSLA'}
✅ Data pulled successfully. Showing top 5 rows:


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
None,TSLA,Common Stock,Tesla Inc,"Tesla, Inc. is an American electric vehicle an...",1318605,NASDAQ,USD,USA,MANUFACTURING,MOTOR VEHICLES & PASSENGER CAR BODIES,...,9.34,65.61,2.58,488.54,167.41,270.59,291.34,3220960000,None,None




⚡ Prompt 2: Get me Microsoft's ESG disclosure metrics
Parsed prompt: {'type': 'fundamental', 'dataset_type': 'overview', 'ticker': 'MSFT'}
✅ Data pulled successfully. Showing top 5 rows:


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
None,MSFT,Common Stock,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,...,11.09,20.32,1,465.64,344.79,387.24,415.37,7433980000,2025-06-12,2025-05-15




⚡ Prompt 3: Show me Apple's environmental, social, and governance scores
Parsed prompt: {'type': 'fundamental', 'dataset_type': 'overview', 'ticker': 'AAPL'}
✅ Data pulled successfully. Showing top 5 rows:


...



⚡ Prompt 7: Fetch the full ESG metrics of Nvidia
Parsed prompt: {'type': 'fundamental', 'dataset_type': 'overview', 'ticker': 'NVDA'}
✅ Data pulled successfully. Showing top 5 rows:


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
None,NVDA,Common Stock,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,...,20.5,31.06,1.958,153.12,81.23,115.25,125.57,24399999000,2025-04-02,2025-03-12




⚡ Prompt 8: What is the ESG performance of Netflix?
❌ Error occurred: Invalid LLM response, missing 'type' field. Full output:
{}


⚡ Prompt 9: Retrieve Salesforce's ESG scorecard
Parsed prompt: {'type': 'fundamental', 'dataset_type': 'overview', 'ticker': 'CRM'}
✅ Data pulled successfully. Showing top 5 rows:


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
None,CRM,Common Stock,Salesforce.com Inc,"Salesforce.com, Inc. is an American cloud-base...",1108524,NYSE,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,...,6.72,22.86,1.381,368.0,210.77,278.4,292.52,961000000,2025-04-24,2025-04-10




⚡ Prompt 10: Get the latest ESG ratings for Adobe
Parsed prompt: {'type': 'fundamental', 'dataset_type': 'overview', 'ticker': 'ADBE'}
❌ Error occurred: We have detected your API key as 0S010WPZM6ECHSRF and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.
